In [11]:
import pandas as pd
import os
import pickle
import numpy as np

# Chemin du fichier Parquet avec les données clean (177 recettes)
file_path = "../data/cleaned_data.parquet"

# Charger les données clean
df = pd.read_parquet(file_path)

# Charger fichier embeddings des données clean
with open("../data/embeddings.pkl", "rb") as f:
    embeddings = pickle.load(f)

In [12]:
# #Ou fichier avec toutes les recettes  : (2676 recettes)

# file_path = "../data/recettes_rag.csv"

# df = pd.read_csv(file_path, encoding="utf-8", sep=";")

# #et charger les embeddings de toutes les recettes
# #Chemin du fichier d'embeddings
# embedding_path = "../data/recettes_embeddings.parquet"
# # Charger les embeddings
# df_embeddings = pd.read_parquet(embedding_path)
# # Convertir la colonne "embeddings" en `numpy.array`
# embeddings = np.array(df_embeddings["embeddings"].tolist(), dtype=np.float32)

In [18]:
len(embeddings)

2676

In [3]:
from sentence_transformers import SentenceTransformer
import numpy as np

# Charger un modèle d'embedding
embedding_model = SentenceTransformer('dangvantuan/french-document-embedding', trust_remote_code=True)

c:\Users\Ihnhn\anaconda3\envs\env_td_llm\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# # Générer les embeddings 1 fois puis enregistrer fichier

# Ajouter une colonne combinée pour les embeddings
#df["combined_text"] = df["Titre"] + " - " + df["Infos"] + " - " + df["Ingrédients"] + " - " + df["Instructions"] + " - " + df["Temps de préparation"] + " - " + df["Infos régime"] + " - " + df['Valeurs pour 100g'] + " - " + df['Valeurs par portion']

# embeddings = embedding_model.encode(df["combined_text"].tolist(), convert_to_numpy=True)
# # Vérifier la dimension des embeddings
# print(f"Dimension des embeddings générés : {embeddings.shape[1]}")  # 768

# import pickle
# # Sauvegarde des embeddings
# with open("embeddings.pkl", "wb") as f:
#     pickle.dump(embeddings, f)

In [15]:
import chromadb

# Initialiser ChromaDB (avec persistance locale)
chroma_client = chromadb.PersistentClient(path="./chroma_db")

# Supprimer l'ancienne collection si elle existe
try:
    chroma_client.delete_collection("recettes")
    print("✅ Collection 'recettes' supprimée avec succès.")
except:
    print("⚠️ Aucune collection 'recettes' à supprimer, on continue...")

✅ Collection 'recettes' supprimée avec succès.


In [16]:
# Créer une nouvelle collection avec la bonne dimension
collection = chroma_client.get_or_create_collection(name="recettes", metadata={"dimension": 768})

print("✅ Nouvelle collection créée avec dimension 768.")

✅ Nouvelle collection créée avec dimension 768.


In [17]:
# Ajouter les recettes dans ChromaDB
for i, (embedding, row) in enumerate(zip(embeddings, df.iterrows())):
    _, row_data = row
    collection.add(
        ids=[str(i)],  # ID unique
        embeddings=[embedding.tolist()],  # Embedding sous forme de liste
        metadatas=[{ 
            "Titre": row_data["Titre"],
            "Temps de préparation": row_data["Temps de préparation"],
            "Ingrédients": row_data["Ingrédients"],
            "Instructions": row_data["Instructions"],
            "Infos régime": row_data["Infos régime"],
            "Valeurs pour 100g": row_data["Valeurs pour 100g"],
            "Valeurs par portion": row_data["Valeurs par portion"]
        }]
    )

print("✅ Tous les embeddings ont été ajoutés dans ChromaDB !")

✅ Tous les embeddings ont été ajoutés dans ChromaDB !


In [19]:
def search_recipe(query, top_k=3):
    query_embedding = embedding_model.encode([query])[0].tolist()
    results = collection.query(
        query_embeddings=[query_embedding],
        n_results=top_k
    )
    return results["metadatas"][0] if results["ids"] else []

In [20]:
#chercher recettes
query = "Gateau chocolat fondant"
recipes_found = search_recipe(query)

for recipe in recipes_found:
    print(f"🔹 {recipe['Titre']}\n🥦 Ingrédients: {recipe['Ingrédients']}\n📖 Instructions: {recipe['Instructions']}\n")

🔹 Fondant au Chocolat, des Commis
🥦 Ingrédients: Beurre demi-sel : 125 g; Sucre : 100 g; Poudre d’amande : 70 g; Chocolat noir : 160 g; Farine de blé : 120 g
 Laissez-le tiédir avant de lengloutir !2 à 5 min selon votre four.joutez les œufs 1 par 1, en mélangeant bien au fouet entre chaque. | 4. Sur plaque : Chemisez* un moule rond à bord haut denviron 20 à 25 cm de diamètre et disposez-le sur une plaque de cuisson. Ainsi, pas daccident de parcours entre le plan de travail et le four ! | 5. Ultime étape : Versez la pâte dans le moule à laide dune maryse. Ne mangez pas tout à même le saladier ! Enfournez-le dans le four chaud pour 18 min. Pour vérifier la cuisson, plantez un couteau dans le gâteau , la lame doit ressortir humide. | 6. Instant hautement chocolaté : À la sortie du four, le gâteau peut être légèrement tremblotant.

🔹 Gâteau au Chocolat, Noir, Cœur Coulant au Chocolat Blanc
🥦 Ingrédients: Chocolat blanc : 90 g; Crème fraîche : 70 g; Chocolat noir : 100 g; Beurre doux : 110 

In [21]:
from mistralai import Mistral

# Charger l'API Mistral
api_key = "z5bIFnE1fro0EjEH2Dm68akoLmPKBIjv"  # Clé API
mistral_client = Mistral(api_key=api_key)


def generate_response(query):
    # 🔹 Rechercher une recette pertinente avec ChromaDB
    recipes = search_recipe(query, top_k=1)

    # 🔹 Si aucune recette 
    if not recipes:
        print("🚫 Aucune recette trouvée. Mistral NE répond pas.")
        return "Désolé, je ne peux répondre qu'avec les recettes disponibles dans ma base."

    # 🔹 Construire le contexte avec la recette trouvée
    recipe = recipes[0]
    context = f"""Voici une recette basée sur ta recherche :
    **Nom :** {recipe['Titre']}
    **Temps de préparation :** {recipe['Temps de préparation']}
    **Ingrédients :** {recipe['Ingrédients']}
    **Instructions :** {recipe['Instructions']}
    **Valeurs nutritionnelles (100g) :** {recipe['Valeurs pour 100g']}
    **Valeurs nutritionnelles (par portion) :** {recipe['Valeurs par portion']}"""

    # 🔹 Générer une réponse avec Mistral
    chat_response = mistral_client.chat.complete(
        model="mistral-large-latest",  
        messages=[
            {"role": "system", "content": "Tu es un assistant culinaire qui propose des recettes healthy disponibles dans la base de données."},
            {"role": "assistant", "content": context},
            {"role": "user", "content": query},
        ],
        temperature=0.1,  # Valeur entre 0 et 1 (plus haut = plus créatif)
        top_p=0.9,        # On garde les 90% des mots les plus probables
        # max_tokens=512,   # Limite la longueur de la réponse
        presence_penalty=0.0,  # 0 = neutre, >0 = pousse à explorer d'autres sujets
        frequency_penalty=0.0  # 0 = neutre, >0 = réduit la répétition
    )

    return chat_response.choices[0].message.content

In [23]:
#query = "Fais moi un programme pour préparer un marathon"
query = "Donne moi une recette de Gâteau Onctueux, Poire Chocolat"
response = generate_response(query)
print(response)

Bien sûr ! Voici une recette de Gâteau Onctueux, Poire Chocolat :

### Gâteau Onctueux, Poire Chocolat

#### Temps de préparation :
25 minutes

#### Ingrédients :
- Poires : 3 pièces
- Chocolat noir : 250 g
- Beurre : 80 g
- Sucre : 150 g
- Poudre d’amandes : 80 g
- Farine de blé : 30 g
- Levure chimique : 1 sachet
- Crème liquide : 100 g

#### Instructions :

1. **Préparation des poires au sirop :**
   - Sortez le beurre du frigo pour qu'il ramollisse.
   - Dans une casserole, faites bouillir de l'eau avec 70 g de sucre.
   - Épluchez les poires en gardant la queue.
   - Plongez les poires dans le sirop et laissez-les cuire 10 minutes à feu moyen.
   - Laissez-les ensuite tiédir hors du sirop. Gardez le sirop pour parfumer une salade de fruits.

2. **Préparation du chocolat :**
   - Préchauffez le four à 180°C en chaleur tournante.
   - Faites fondre 150 g de chocolat au bain-marie ou quelques secondes au micro-ondes. Laissez tiédir le chocolat hors du feu et gardez la casserole pour 